<a href="https://colab.research.google.com/github/szsctt/learn_torch/blob/main/learn_torch_3_neural_nets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building neural nets

From the [torch documenation](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html).

We use the `nn` module from `torch` to build neural networks.

Here this is demonstrated by using the FashionMNIST dataset.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Choosing a device to use

Training on a GPU is generally faster, so use it if if one is available.

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

Using cuda


### Define a network class

We define a class by subclassing `nn.Module`, and initalize the layers in the `__init__` method.  

We also have to define a `forward` method that defines how to do forward propagation.

In [3]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits


We create an instance of `NeuralNetwork` and move it to `device`.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Use a model by passing it data, which calls its `forward` method as well as doing some other stuff.  This other stuff can be important, so avoid calling the `forward` method directly.

In [5]:
X = torch.rand(1, 28, 28 , device=device)
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print(f"Predicted class {y_pred}")


Predicted class tensor([2], device='cuda:0')


### Model layers

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten()

We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained).

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear()

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU()

Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1337,  0.4827,  0.5172,  0.0405, -0.0777,  0.3644, -0.4315, -0.1168,
          0.0867,  0.6330, -0.8507, -0.1167, -0.0566,  0.0995, -0.0707,  0.1849,
          0.1589, -0.4606, -0.4586, -0.1069],
        [-0.0583,  0.1156,  0.7601, -0.2639,  0.1834,  0.6562, -0.3446, -0.1370,
          0.0266,  0.3410, -0.4858,  0.0395, -0.1243, -0.1980,  0.0618,  0.4563,
          0.1287, -0.6040, -0.1754, -0.0533],
        [-0.0579,  0.5081,  0.4136, -0.2245, -0.0844,  0.5099, -0.4828, -0.1277,
          0.0718,  1.1178, -0.6548,  0.0813,  0.0343,  0.1294, -0.2276,  0.3000,
          0.2848, -0.3334, -0.2636, -0.2942]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.4827, 0.5172, 0.0405, 0.0000, 0.3644, 0.0000, 0.0000, 0.0867,
         0.6330, 0.0000, 0.0000, 0.0000, 0.0995, 0.0000, 0.1849, 0.1589, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.1156, 0.7601, 0.0000, 0.1834, 0.6562, 0.0000, 0.0000, 0.0266,
         0.3410, 0.0000, 0.0395, 0.0000, 0.0000, 0.06

#### nn.Sequential()

nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#### nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

#### Model Parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name}; size: {param.size()}; Values: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight; size: torch.Size([512, 784]); Values: tensor([[ 0.0177,  0.0167,  0.0193,  ...,  0.0141, -0.0186,  0.0290],
        [ 0.0206,  0.0117,  0.0330,  ...,  0.0352, -0.0176, -0.0026]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias; size: torch.Size([512]); Values: tensor([0.0302, 0.0136], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight; size: torch.Size([512, 512]); Values: tensor([[ 0.0259, -0.0077, -0.0061,  ..., -0.0168, -0.0129,  0.0150],
        [ 0.0134, -0.0372,  0.0132,  ...,  0.0358, -0.0099,  0.0254]],
       device='cuda:0', grad_fn=<SliceBackward0>